# Loading NIfTI files and aligning with event data

I'm using VOT distributional learning data as an example here.  The goal is to load in event data to a data frame, and the NIfTI files to another data frame (keyed by subject and run number).  Then split up the behavioral data to create a separate alignment for each run/subject combination.  Finally, concatenate the alignments, NIfTI files, and the event data frames together into a combined dataset.

The goal is to have three things:

1. An event data frame with $n$ rows that describes all the relevant events
2. A big 4d $x,y,z,t$ NIfTI array where $t=1..T$
3. A n×T (sparse) matrix where each row corresponds to an event and each column to a time slice in the NIfTI data.  The entry at (i,t) corresponds to the HRF for event i evaluated at time t.

## Loading behavioral data

In [5]:
using YAML, DataFrames, CSV

INFO: Recompiling stale cache file /home/dave/.julia/lib/v0.6/CSV.ji for module CSV.


In [3]:
vot_dir = "/home/dave/work/experiments/adapt-fmri/imaging_analysis"

clean_octals(s) = replace(s, r"subject: 0+", "subject: ")

sessions = joinpath(vot_dir, "session_info.yml") |> readstring |> clean_octals |> YAML.load
sess = sessions[1]

Dict{Any,Any} with 5 entries:
  "image_directory"  => "data-imaging/vot-adapt03/20150430"
  "bold_images"      => String["data-imaging/vot-adapt03/20150430/vot-adapt03_2…
  "anatomical_image" => "data-imaging/vot-adapt03/20150430/vot-adapt03_20150430…
  "behavioral_data"  => "data-behav/vot_adapt_003_scan.csv"
  "subject"          => 3

In [9]:
load_behav(s) = CSV.read(joinpath(vot_dir, sess["behavioral_data"]), nullable=true)

behav = load_behav(sess)
#behav[:start_secs_relative] = 0.
#by(behav, :run_number) do b
#    b[:, :start_secs_relative] = b[:start_secs] - minimum(b[:start_secs])
#    b
#end

,experiment_name,expt_start_date_time,subject_num,subject_handedness,run_number,scan_or_behav,b_vot_cond,trial_number,category,vot,word,picture_left,picture_right,response,response_keycode,response_key,rt,sound_on_secs,pic_on_secs,start_secs,start_date_time,jitter_trs
1,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,1,p,20,BEES20.WAV,bees.png,peas.png,right,31,2@,1.711,0.15,0.006,1992.96,30-Apr-2015 14:06:37,2
2,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,2,b,-10,BEES-10.WAV,peas.png,bees.png,right,31,2@,1.695,0.15,0.002,1998.96,30-Apr-2015 14:06:42,2
3,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,3,p,10,BEACH10.WAV,peach.png,beach.png,left,30,1!,1.095,0.15,0.007,2004.96,30-Apr-2015 14:06:48,3
4,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,4,p,50,BEES50.WAV,peas.png,bees.png,left,30,1!,1.391,0.15,0.009,2013.96,30-Apr-2015 14:06:54,1
5,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,5,p,30,BEACH30.WAV,peach.png,beach.png,left,30,1!,1.311,0.15,0.007,2016.96,30-Apr-2015 14:07:03,2
6,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,6,b,-10,BEACH-10.WAV,peach.png,beach.png,right,31,2@,1.278,0.15,0.003,2022.96,30-Apr-2015 14:07:06,1
7,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,7,p,40,BEAK40.WAV,beak.png,peak.png,right,31,2@,1.927,0.15,0.001,2025.96,30-Apr-2015 14:07:12,2
8,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,8,b,-10,BEACH-10.WAV,beach.png,peach.png,left,30,1!,1.135,0.15,0.006,2031.96,30-Apr-2015 14:07:15,2
9,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,9,b,-10,BEAK-10.WAV,peak.png,beak.png,right,31,2@,1.663,0.15,0.002,2037.96,30-Apr-2015 14:07:21,1
10,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,10,b,-10,BEES-10.WAV,bees.png,peas.png,left,30,1!,1.135,0.15,0.008,2040.96,30-Apr-2015 14:07:27,2


## Load NIfTI

In [14]:
using NIfTI

function add_prefix(str, prefix)
    path, fn = splitdir(str)
    joinpath(path, prefix * fn)
end

read_one_nifti(fn, prefix; kwargs...) = niread(joinpath(vot_dir, add_prefix(fn, prefix)); kwargs...)

function load_nifti(s; prefix="wra")
    niftis =  read_one_nifti.(s["bold_images"], [prefix])
    return DataFrame(run_number=collect(1:length(niftis)), subject_num=s["subject"], nifti=niftis)
end

INFO: Recompiling stale cache file /home/dave/.julia/lib/v0.6/MappedArrays.ji for module MappedArrays.


load_nifti (generic function with 1 method)

One issue with this particular dataset is that SPM for some reason strips off timing information from where NIfTI.jl expects to find it in the header.

In [15]:
niftis_preproc = read_one_nifti.(sess["bold_images"][1:1], ["", "a", "ra", "wra"], mmap=true)
headers = [ni.header for ni in niftis_preproc]
voxel_size.(headers)
time_step.(headers)

4-element Array{Float64,1}:
 3012.5
    0.0
    0.0
    0.0

Because the dataset itself isn't critical, just use the un-preprocessed NIfTI files.

In [16]:
run_niftis = load_nifti(sess, prefix="")
names(run_niftis)

3-element Array{Symbol,1}:
 :run_number 
 :subject_num
 :nifti      

In [17]:
map(size, run_niftis[:nifti])

4-element Array{NTuple{4,Int64},1}:
 (64, 64, 30, 116)
 (64, 64, 30, 116)
 (64, 64, 30, 116)
 (64, 64, 30, 116)

In [18]:
[time_step(ni.header) for ni in run_niftis[:nifti]]

4-element Array{Float64,1}:
 3012.5
 3012.5
 3012.5
 3012.5

In [19]:
behav_by_run = groupby(behav, :run_number)
behav_by_run[2]

,experiment_name,expt_start_date_time,subject_num,subject_handedness,run_number,scan_or_behav,b_vot_cond,trial_number,category,vot,word,picture_left,picture_right,response,response_keycode,response_key,rt,sound_on_secs,pic_on_secs,start_secs,start_date_time,jitter_trs
1,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,1,p,30,BEAK30.WAV,peak.png,beak.png,right,31,2@,1.535,0.15,0.009,2388.02,30-Apr-2015 14:13:12,1
2,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,2,b,0,BEES0.WAV,bees.png,peas.png,left,30,1!,0.863,0.15,0.007,2391.02,30-Apr-2015 14:13:17,2
3,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,3,b,-20,BEAK-20.WAV,peak.png,beak.png,right,31,2@,1.431,0.15,0.003,2397.02,30-Apr-2015 14:13:20,2
4,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,4,p,30,BEAK30.WAV,beak.png,peak.png,left,30,1!,1.007,0.15,0.008,2403.02,30-Apr-2015 14:13:26,1
5,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,5,b,-10,BEES-10.WAV,bees.png,peas.png,left,30,1!,0.967,0.15,0.006,2406.02,30-Apr-2015 14:13:32,2
6,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,6,b,-10,BEACH-10.WAV,peach.png,beach.png,right,31,2@,1.623,0.15,0.002,2412.02,30-Apr-2015 14:13:35,1
7,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,7,p,40,BEES40.WAV,peas.png,bees.png,left,30,1!,1.167,0.15,0.008,2415.02,30-Apr-2015 14:13:41,2
8,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,8,b,-10,BEES-10.WAV,peas.png,bees.png,right,31,2@,1.519,0.15,0.004,2421.02,30-Apr-2015 14:13:44,1
9,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,9,b,-10,BEAK-10.WAV,beak.png,peak.png,left,30,1!,1.343,0.15,0.002,2424.02,30-Apr-2015 14:13:50,2
10,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,10,b,-10,BEAK-10.WAV,peak.png,beak.png,right,31,2@,1.111,0.15,0.007,2430.02,30-Apr-2015 14:13:53,2


In [20]:
gg = groupby(behav[end:-1:1,:], [:run_number, :subject_num])
vcat((g[1, gg.cols] for g in gg)...)

,run_number,subject_num
1,4,3
2,3,3
3,2,3
4,1,3


In [21]:
function nest(df::DataFrame, cols::Vector{Symbol}; data_col=:data)
    grouped = groupby(df, cols)
    nested = vcat((g[1, cols] for g in grouped)...)
    nested[data_col] = convert(Vector{AbstractDataFrame}, collect(grouped))
    return nested
end

nest (generic function with 1 method)

In [22]:
nested_by_run = nest(behav, [:run_number, :subject_num])
#join(run_niftis, nested_by_run, on = [:run_number, :subject_num])

,run_number,subject_num,data
1,1,3,"55×22 DataFrames.SubDataFrame{Array{Int64,1}}. Omitted printing of 19 columns │ Row │ experiment_name │ expt_start_date_time │ subject_num │ ├─────┼─────────────────┼──────────────────────┼─────────────┤ │ 1 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 2 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 3 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 4 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 5 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 6 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 7 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 8 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 9 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 10 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 11 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ ⋮ │ 44 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 45 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 46 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 47 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 48 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 49 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 50 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 51 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 52 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 53 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 54 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 55 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │"
2,2,3,"56×22 DataFrames.SubDataFrame{Array{Int64,1}}. Omitted printing of 19 columns │ Row │ experiment_name │ expt_start_date_time │ subject_num │ ├─────┼─────────────────┼──────────────────────┼─────────────┤ │ 1 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 2 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 3 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 4 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 5 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 6 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 7 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 8 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 9 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 10 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 11 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ ⋮ │ 45 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 46 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 47 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 48 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 49 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 50 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 51 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 52 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 53 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 54 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 55 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 56 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │"
3,3,3,"56×22 DataFrames.SubDataFrame{Array{Int64,1}}. Omitted printing of 19 columns │ Row │ experiment_name │ expt_start_date_time │ subject_num │ ├─────┼─────────────────┼──────────────────────┼─────────────┤ │ 1 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 2 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 3 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 4 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 5 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 6 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 7 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 8 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 9 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 10 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 11 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ ⋮ │ 45 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 46 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 47 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 48 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 49 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 50 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 51 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 52 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 53 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 54 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 55 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 56 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │"
4,4,3,"55×22 DataFrames.SubDataFrame{Array{Int64,1}}. Omitted printing of 19 columns │ Row │ experiment_name │ expt_start_date_time │ subject_num │ ├─────┼─────────────────┼─────────

# Connecting events and NIfTIs

Next step is to construct a (sparse) matrix that can transform the events model matrix to the fMRI time series.  Will be event-by-TR.  The first thing we need is a hemodynamic response function (HRF)


## Canonical HRF

SPM uses a canonical HRF that's two gamma PDFs, one for the initial rise and another for the undershoot.  The SPM implementation uses the $\alpha, \beta$ parametrization, while Julia's gamma uses $\alpha, \theta=1/\beta$.  From what I can tell, the parameters are $\alpha=6$ for the peak, $\alpha=16$ for the undershoot, and $\beta$ is set to make the scale 1 second, and the undershoot is scaled by $1/6$.

When generating the HRF, by default it generates 16 frames per TR, and then subsamples back to the correct temporal resolution.  I'm not sure why; presumably because they want to use fractional offsets, but it still doesn't make sense.

The whole thing is normalized to sum to 1, but I'm not going to bother with that...

In [23]:
using Distributions

INFO: Recompiling stale cache file /home/dave/.julia/lib/v0.6/Distributions.ji for module Distributions.


In [24]:
struct HRF{T,S}
    peak::Gamma{T}
    undershoot::Gamma{T}
    undershoot_scale::S
end

In [25]:
(h::HRF)(t) = pdf(h.peak, t) - pdf(h.undershoot, t) * h.undershoot_scale
(h::HRF){T<:Real}(t::T, t_offset::T) = h(t-t_offset)

In [26]:
canonical_hrf = HRF(Gamma(6.0, 1.0), Gamma(16.0, 1.0), 1/6)

# check offset
canonical_hrf.(0:3:32) == canonical_hrf.(3:3:35, 3)


true

## A single run


Let's start by looking at one run.

In [27]:
function make_tr_to_event_mat(ni::NIVolume, events::AbstractDataFrame, time_col::Symbol; hrf::HRF=canonical_hrf)
    n_t = size(ni, 4)
    TR = time_step(ni.header) / 1000
    tr_ts = TR * range(0., n_t)

    event_ts = events[time_col]
    event_ts -= minimum(event_ts)

    # HACK. specify this somewhere...in the canonical HRF?
    hrf_len_s = 32

    sparse(Float64[(event_t <= tr_t <= event_t+hrf_len_s) ? canonical_hrf(tr_t-event_t) : 0
                   for tr_t in tr_ts, event_t in event_ts])
end

events = nested_by_run[1, :data]
ni = run_niftis[1, :nifti]

tr_by_event = make_tr_to_event_mat(ni, events, :start_secs)

116×55 SparseMatrixCSC{Float64,Int64} with 600 stored entries:
  [2  ,   1]  =  0.101658
  [3  ,   1]  =  0.159794
  [4  ,   1]  =  0.0564006
  [5  ,   1]  =  0.000158514
  [6  ,   1]  =  -0.0152137
  [7  ,   1]  =  -0.0127044
  [8  ,   1]  =  -0.00639594
  [9  ,   1]  =  -0.0023371
  [10 ,   1]  =  -0.000672901
  [11 ,   1]  =  -0.000160727
  ⋮
  [114,  54]  =  -0.00424808
  [115,  54]  =  -0.00138701
  [116,  54]  =  -0.000365926
  [109,  55]  =  0.00968648
  [110,  55]  =  0.16783
  [111,  55]  =  0.113119
  [112,  55]  =  0.0241093
  [113,  55]  =  -0.010118
  [114,  55]  =  -0.0152973
  [115,  55]  =  -0.00979193
  [116,  55]  =  -0.00421316

In [28]:
using Plots
gr()

Plots.GRBackend()

In [29]:
plot(tr_by_event)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip9102)" style="stroke:#00a8cb; stro

In [30]:
heatmap(tr_by_event)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
<image width="485" height="364" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAeUAAAFsCAYAAADyj6FyAAASv0lEQVR4nO3dfbCmdVkH8N/Zc9jd
c1h2l911z77vAsvLblEJDebb6DgNrzJqWOAkokEISpAJKBoqMkmNZOVMOUlKRgaYkGKa0WS5ogRD
VlOTZmMjCFomLwu7nH3OeV76pz+Yie7rx55nz7me5/l8/r2/c+8juFxzXffvvu6xNVMn9QoAMC+9
0g0zs539jdeX9OvHAADzoygDQBKKMgAkMbHYPwAAhkGv1wkzrfZjjdd1ygCQhKIMAEkYXwNAKH57
eK47E2Y6nb2N13XKAJCEogwASRhfAzDi4tF0ryLTaj8eZsbHVzZe1ykDQBKKMgAkYXwNwBDrz2i6
ZjFIdLK6lFJOX3Fx43WdMgAkoSgDQBJjvqcMwOBqLmH9Gk3XLAapOX39+FVLG6/rlAEgCUUZAJJw
+hqAgRWNpxdyNH3q5LlhZvKGFzVe1ykDQBKKMgAkYXwNwMCKxtP9Gk3XLAb51OV3hJndR3y/8bpO
GQCSUJQBIAlFGQCSsNELgKQqvmHcear5ep+eF4+PrwozM7MfCjPTR/xO43WdMgAkoSgDQBJeiQIg
pZqPSbTajzVe73SeDO9RM5o+ffK8MLN71Z1hJqJTBoAkFGUASML4GoBF0J/vHEfj6ZrR9BlT8Wj6
trd9KsxsvXFrmInolAEgCUUZAJIwvgagzypG0xWZmo9JjI+vbrx+xlT8jePbf+X2MDNx7a+HmXLj
78WZgE4ZAJJQlAEgCeNrAPqqZjRdc7I6WgxSSilnTr2u8fptb78tvMf4u28IMyes/kyY6QedMgAk
oSgDQBI+3QjAcxCXjG6vHWZqTlbvb30nzDx93crG60ve9YHwHruP/FyYeaL7/TDTDzplAEhCUQaA
JIyvAajWK90wM9vZF2ZqTlbX9I37Zt7feH3Xmj8P77G3+58Vv2Vh6JQBIAlFGQCSUJQBIAkbvQD4
Xwu3iasdfAe5lFJefcQlYWbXmubXmfZ2/yu8RyY6ZQBIQlEGgCS8EgUwMpr/c1/zIYma151qNnFN
jB8ZZp66bi7MbP7A0WFmkOiUASAJRRkAkjC+BhgKFSeno/F1xcnqfbOPVPyW8TBx9uHnhplP/mBn
mNmy5o8qfs/g0CkDQBKKMgAkYXkIwBCoOTkdjadrvnFcs/TjNSvjpR+3vvOWMHP8uheEmWGjUwaA
JBRlAEjC6WuAIdDtxYs25rpPN15vtR8P7zE2Fp+sfur6Vpjp/PK1YWbr2lvDzLDRKQNAEooyACTh
9DVAevFTxmg0XUo8nq45Wf0zKy8NM+0rtoSZ3c+7O8yMIp0yACShKANAEsbXAMnVLAapOTkdjacP
m1gT3uNP3n1zmDl2/U+Fmf3dx8LMKNIpA0ASijIAJKEoA0ASnikDLKr5f0iilLrXmaJnxq+q+Mbx
3GUbwsz+6+4IMzw7nTIAJKEoA0ASxtcAi6jmdaeabV01rzNF4+lPXvvx8B7HTI/eN44Xkk4ZAJJQ
lAEgCeNrgEMmHk13K05Wt9rx9qvXrHh9mPnj93ys+c+5+OrwHk+/564ww8HTKQNAEooyACQxtmbq
pHi+AsBz1ivdMDPbeSrM7G89GGZaH1oaZy66svH6rk1fCe8x090bZjh4OmUASEJRBoAknL4GOGjN
T/9qTlYfqDhZfdjE2jDTuvCiMLNr057G6zPdeH82h5ZOGQCSUJQBIAnja4CDFO2tblfsrO5UnL4+
Z/WlYeaEzfeEmQPd+M9icemUASAJRRkAkrA8BOAgdXpzjdf3zT4c3mN8bFmY2febT4SZDdfsDjPk
p1MGgCQUZQBIwulrgGcVP9mLTlfXnKx+7er4k4sHzo9H3OWaL8UZ0tMpA0ASijIAJKEoA0ASnikD
PItoW1cp8ccklk6sC+9xy/U3hZnt204JMwwHnTIAJKEoA0ASxtfACIpH0zXfQo5eefrZmtedXhf3
Rq13fCXMMBx0ygCQhKIMAEn4IAUwcnqlG2ZmK7ZxzXWaN3rt+/Dj4T22Xf2T8W+p+C4zw0GnDABJ
KMoAkITT18DIqTlZHS0GKaWUc1ee33yPc5q/t1xKKbNX3hdmGB06ZQBIQlEGgCSMr4EhM//vIJdS
9y3kT9zw0cbrW445ObwHPJNOGQCSUJQBIAnLQ4ABEv/nquaTi0+2Hgwz40uWh5lHHzqr8fr2Y74e
3gOeSacMAEkoygCQhNPXwMCoGU3345OLpZRyXrAYpJRSdu58IMzAc6FTBoAkFGUASEJRBoAkvBIF
DIxObzbMzNVs6+oeCDP7PhJ/kGL68t1hBp4LnTIAJKEoA0ASXokCkoifpNWMplsV30F+3ao3hJkD
p8f3KeVfKzJQT6cMAEkoygCQhNPXQAq90g0zT7a+E2Y6nX1hZu7m+Mnd5sueH2bavVaYgedCpwwA
SSjKAJCE09dACt1eO8zUjKaXHTYdZg684iVhpt379zAD/aZTBoAkFGUASML4GlggzS961CwGqRlN
n3vEOWHmmN33hxlYDDplAEhCUQaAJIyvgQXRC8bXNTurX7863ln9sd+6Kcysv2RXmIHFoFMGgCQU
ZQBIwu5rYEF0erON1/fOfDO8R/uWpWHmwItOCzPbTnwwzMBi0CkDQBKKMgAkoSgDQBJeiQL6ID6a
Em3sWnbYhvAerReeFGaO+rHvhRnISqcMAEkoygCQhPE1MG/Rtq5SSmnNNW/sOv/IeFvXjh+/L8x0
enNhBrLSKQNAEooyACRhoxcwb9G2rlLijV3tWyfDe6y/8Njq3wSDSKcMAEkoygCQhNPXQGD+i0FK
KWX5YRsbr88+f3fFb/nvigwMLp0yACShKANAEsbXMNLi0XQ/FoOUUsoFa5qXg2w/+e/Ce8Cw0ykD
QBKKMgAkYXkIjLBe6YaZbq8dZqLFIKWU0v7TFY3X119wdHgPGHY6ZQBIQlEGgCScvoahFT+ZqhlN
Vy0GWdq8GKSUUuZ27QwSe8N7wLDTKQNAEooyACShKANAEp4pw5Cq2cRV87y4NfdomHnT2gvCzNZT
7g0zMOp0ygCQhKIMAEnY6AVDqtObDTP7Wt+N79PdH2baf7Y6zKz/+W1hBkadThkAklCUASAJp69h
YDU/eZqrGDvXjKYnl24KM+0dW8JMKQcqMjDadMoAkISiDABJGF/DgIqWg7TmHgvvMbl0c5g5f9Wr
wsyWF1sMAv2gUwaAJBRlAEjC+BoGVPQt5JqT1W9YHe+s/t0/uD3MfPq8eAwOxHTKAJCEogwASdh9
DSnFfy0PtJ8I7tAN7/HUna0w01kfj6Y3vtx/RqAfdMoAkISiDABJOH0NCUWLQUoppdVuXg7yi+vi
k9WddXvCzKaXj4WZmnE7ENMpA0ASijIAJOH0NSy4+K9cpzcXZvbOfLPx+twXpsN7bHjtxjBTc4ob
6A+dMgAkoSgDQBKKMgAk4ZUo6Kv4eXHN605znfhjEpNLtzT/OStXVPwWz4shE50yACShKANAEsbX
UK0/o+noO8ilxNu6Sinlzc9r3ti18dSvhvewiQty0SkDQBKKMgAkYaMXVKo5qVwzmq45Wf307Hfj
+9zd/J3j6VevDe8B5KJTBoAkFGUASMLpa6jUr9F0zcnqqaVbw0zvMH99YdjolAEgCUUZAJIw/4JS
Ss0SjX6Npjvd+D6/sKZ5MUgppWw8K1oO4s

In [33]:
using StatsModels, CategoricalArrays
categorical!(behav, :category)

mm = ModelMatrix(ModelFrame(Formula(nothing, :(0 + category)), events))
mm.m

55×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 ⋮       
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 1.0  0.0
 0.0  1.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 1.0  0.0

In [35]:
plot(tr_by_event * mm.m)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 y2

In [37]:
plot(ni[32, 32, 15, :])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 440 
 
 
 450 
 
 
 460 
 
 
 470 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [39]:
using GLM

INFO: Precompiling module GLM.


In [40]:
X = [ones(size(tr_by_event, 1)) tr_by_event * mm.m]
y = convert(Vector{Float64}, ni[32, 32, 15, :])

fitted = lm(X, y)


GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}}:

Coefficients:
     Estimate Std.Error   t value Pr(>|t|)
x1    452.829   1.35432   334.359   <1e-99
x2   -4.15423    10.475 -0.396585   0.6924
x3    27.0898   10.2266   2.64895   0.0092



Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] ccdf(::Distributions.FDist{Float64}, ::Array{Float64,1}) at ./deprecated.jl:57
 [3] coeftable(::GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}}) at /home/dave/.julia/v0.6/GLM/src/lm.jl:154
 [4] show(::IOContext{Base.AbstractIOBuffer{Array{UInt8,1}}}, ::GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}}) at /home/dave/.julia/v0.6/GLM/src/linpred.jl:138
 [5] limitstringmime(::MIME{Symbol("text/plain")}, ::GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}}) at /home/dave/.julia/v0.6/IJulia/src/inline.jl:24
 [6] display_dict(::GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}}) at /home/dave/.julia/v0.6/IJulia/src/execute_request.jl:27
 [7] exec

In [41]:
plot([y predict(fitted)])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 440 
 
 
 450 
 
 
 460 
 
 
 470 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 y2

### Sparse matrix

Using a sparse matrix for the transformation is a lot faster (which would hopefully be the case given that it's mostly zeros...)

In [42]:
#event_to_tr = Float64[event_t <= tr_t ? canonical_hrf(tr_t-event_t) : 0 for tr_t in tr_ts, event_t in event_ts]
using BenchmarkTools

@benchmark $(full(tr_by_event)) * $(mm.m)

INFO: Recompiling stale cache file /home/dave/.julia/lib/v0.6/BenchmarkTools.ji for module BenchmarkTools.


BenchmarkTools.Trial: 
  memory estimate:  1.98 KiB
  allocs estimate:  1
  --------------
  minimum time:     7.120 μs (0.00% GC)
  median time:      7.379 μs (0.00% GC)
  mean time:        7.759 μs (2.15% GC)
  maximum time:     575.229 μs (96.86% GC)
  --------------
  samples:          10000
  evals/sample:     4

In [43]:
@benchmark $(tr_by_event) * $(mm.m)

BenchmarkTools.Trial: 
  memory estimate:  1.98 KiB
  allocs estimate:  1
  --------------
  minimum time:     1.652 μs (0.00% GC)
  median time:      1.740 μs (0.00% GC)
  mean time:        1.958 μs (8.76% GC)
  maximum time:     221.880 μs (97.47% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [44]:
# row or column major??
full(mm.m' * tr_by_event')' == tr_by_event * mm.m
@benchmark full($(mm.m') * $(tr_by_event'))'

BenchmarkTools.Trial: 
  memory estimate:  3.97 KiB
  allocs estimate:  2
  --------------
  minimum time:     2.458 μs (0.00% GC)
  median time:      2.609 μs (0.00% GC)
  mean time:        3.049 μs (12.50% GC)
  maximum time:     254.160 μs (97.06% GC)
  --------------
  samples:          10000
  evals/sample:     9

## Multiple runs

Generate separate transformers for each run, then combine to create a transformer for all runs.

In [45]:
tr_to_ev = blkdiag(make_tr_to_event_mat.(run_niftis[:nifti], nested_by_run[:data], [:start_secs], hrf=canonical_hrf)...)
heatmap(tr_to_ev)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
<image width="485" height="364" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAeUAAAFsCAYAAADyj6FyAAAgAElEQVR4nO3deZxddX3/8Vdmnywz
mSyzZZkJSQgJivJzwe1nWxURRLFUBCmQCC6AgKAWUdxARflBa9VfWyXFBQQKouKOdUMEjBaXtr9W
xWpZBSoJJJnJzL1zl98f53zPvbmQkGXm3O31/GfunPO9yeehD3jz+X6/53tmLZh9aBFJklQ1hWIO
gLYq1yFJUtObKkwAhrIkSVVTJJqszuQeBQxlSZKqpljMA5DPbwUMZUmSUhc65LHsHwA4cu4bAUNZ
kqTUhDCu7JCvP+vLAMxy97UkSTOrMozDxq6wljyRvRywU5YkacbtKoyP6D4BgHW9NwGGsiRJMyJ0
xwBj2QcAyOe3AdDa2gvAlx59GgBDvXcAhrIkSTMidMdQHsbzAThy9vEArOv76k7fMZQlSZpGoUMO
U9Xw+DC+/q3XA7D0smU7fddQliRpGoSjMkMYj2fuTu7tuKgHgFkXHATAuoXPiu88tNOf4e5rSZKm
QSa/PfqZ2xJfaUnubRt/DwDrFt0c/V7YOYwDO2VJkvZD6ajMKIxz8frxq3pOT8asW/QtALYVHt7t
n2WnLEnSfsjH09ZbJ34FQFtrHwDbL5pKxiy5ZMUe/Vl2ypIk7YPQIY/HjzsdG3fG115wFQAHXnJY
2ehH9ujPtFOWJGkPlT97XIgfeQod8uSHZwMwdc4FAIws/sJe//l2ypIkPYkQxoWyZ49zhR0AtLct
AGDq7HMAWDfwvX3+ewxlSZJ2oTKMQxBDaWPXMXNPAmDt4A8A2FHYwr4ylCVJ2oVsxWNOufh3KHXI
n7kvOhhkdHDfwzgwlCVJqhA65Mk4jPNxGLe3LUzGvGrOawBYO3zbtP29hrIkSbHK6erKMP7zOIgB
rn7flQAMvXvNtP39hrIkqSk90U7qsGYcOuTj558BwNXv3xhdP6Uj+c6K5eGozG3TVpOPREmSmlI4
9AMeH8ahQ85+LOpdJ0+KwnnNyM+S72QKpfXl6WKnLElqSuU7qSvDuKNtUXT9xGi6es3InQBkCmMz
WpOhLElqKpWbuODxYXzs3OMAOHDFLwDIznAYB4ayJKkp5IvRWdTh1Yqts7qSe6/pOxmAz33oCgAy
x2YAGDkgnTAODGVJUlOYiqerM7lHATi+5+Tk3ucuicJ48lV/CcCqVf+RcnURN3pJkhpamK7elrkH
KE1VT32y1JdOvvIEAFat/m10r7iDarBTliQ1tEK8yzpZN25fDMDkKw5PxqxafRcAU8WJlKvbmaEs
SWpIYQ15LHs/ACctOBOAz/yfaKp6YPW/V6ew3XD6WpLUkCbzWwHYkbkXgKkroz4085JjAFi+9q7q
FLYbdsqSpIZRfkpXeIlEZ/tA9PuLnwfAAevuTb+wPWQoS5LqXgjjTL505GVbyxwAxv9xMwD96/4r
/cL2kqEsSap7YTNXZqp0IMhJfadE155/d3zlvpSr2nuuKUuS6l7l+jFA7uro5RFDpz8VgHwxm35h
e8lOWZJUt5Jp67hD7mofTO5lnn0IAPnig+kXto8MZUlS3alcQ25vnQvA2FVbkzH9h9ZPGAeGsiSp
7lSuIa9fEK0fZw8pPx5zS+XXap5rypKkulHZIYc15NwN0U7rwdetScYW4sND6omdsiSpbpQ65Ogx
p66OIQCmDloR399encKmiaEsSap5oUMey0SPNbW39Ua/Xxe9WrH/OfUdxoGhLEmqSeWncyUvlSiM
A3Bq73oAcqM/jUdkUq1tprimLEmqSfmyNeGpfBTGRfIAbL8herXi4ImjABTi6/XOTlmSVFOStztl
SidwhQ4599WFAAycMAqUQrpRGMqSpJoSuuIQxADdHUuia4ujteRGC+PAUJYk1YTkcaf47U7dHUuT
e+vnvxKA4RffEV8xlCVJmjGVm7k29K1P7n38M18E4IZX96dfWIoMZUlSVYTOOIRxWEM+c+BMAP72
+puTsYNHhTOtC+kVWAXuvpYkVUXY0BXWkHdko1Ce+udo/XjwVYuSscUGD+PATlmSlJryZ49DGIc1
5Nkdy6IbLbPisc0RxOUMZUnSjKvsiqHUGc/uXA7A1m9GYTzwip6Uq6sdhrIkacaEzngqHx2HGbpi
KIXx6/uOji5kv5RucTXIUJYkzZjkBRJxGOcLO5J7b1gQhfFl10UbugaPCjurm3erk6EsSZp2oUPO
xh1yCOM5nSPJmMv+6dsAtGzbGn9nTpol1iRDWZI0bUIYT+YeA2Aiez8A5w1FjzldesN3k7GDR3TF
n2anV2CNM5QlSdOmEG/oysbT1XM6RwG49AvfB6Dl4YfKRo+gnRnKkqT9Uv6YUzY5tzqarn7TwqMA
aHnwagD616/Y6ZvamYeHSJL2S76YTT6HU7m62qNNW1tuidaJh17cGo8wcnbHTlmStE9Ch1z+isXu
9gEANt86D4ChPyuN1pMzlCVJe6V0ZnW0flz+mNMZi14GQOtvr4yvrEm1tnrn9LUkaa+E6eqwftza
0pHce+S26H3Hwy8Mr1Y0YvaGnbIkaY+EMB7L3Bv9XpgAYPLnz0rGDL8gE38yjPeFoSxJ2q3SQSBh
Z3UUxnO7VgLQ+vNflI1em2ptjcZQliTtVunZ481AKYzPXvRSAAbOub1stB3y/jCUJUlP6PFHZUYd
cgjj937jlwB86rkG8XQxlCVJiWKxFLCT+UcBmMg+AMCFy84A4H3v2AjA4HPXpVxd43P3tSQpCeMC
U8m1sKFrdscSAB7+2TAAS5/1WPhWegU2CTtlSWpiIYzzcRiH9x5Dabr6Lf2HA9D2nY3xHTdzzRQ7
ZUlqYrn4MacQxuG9xwBzOpYC8PAvop9Ln/FIytU1HztlSWpCoUMOU9SFuCvu6TowGfPQv0ZvcVp6
6EMoHYayJDWhMF2dhHF3FMbn9b8oGdNy02fjT6vTLK2pGcqS1ERChxymq3u6o7Op39ofvTnir757
dzJ2+GmGcdoMZUlqIpWPOT3yu5cD0HLd5wFY8rQVT/xFpcJQlqQGVywWks+Z+FSu3u6DAGi55lYA
lnzEMK4FhrIkNagQxnlyybVCYRKAtw/8CQBLPnJH+oVpl3wkSpIaTGUYT+W3J/c6WnsAeOA/o055
+cG/S7k67Y6dsiQ1mLBuHJ45LsbdMcD/3PNcAJavvSv9wvSkDGVJahChQ64M497u0glcsz75w/jT
UKq1ac8YypJU5yqnq0thHL0w4h2DL0jGLv/Yj1OuTnvDNWVJqnPhqMxsvHbc1TofgAuGojB+863b
krEjB/5HytVpb9gpS1KdCh3yWOae6Pe4Q/7jH6JTuQqXfR2AkQMXVqE67QtDWZLqVDgqM4Tx/O6D
o+uX3gLAARsN43pjKEtSncrGR2XO734KAO8afh4AB2x03bheGcqSVEdyxUzyOZP9AwD/8/BRAExd
9M8AfPiz89IvTNPCjV6SVEd2xMdkAsxui6an7zplHIADr5pTlZo0feyUJamOZMtC+ZLl0cskDrxq
U7XK0TQzlCWpjhQL2eTz6ze1A/DB5dWqRtPNUJakOvLR1acln0eX/6iKlWgmGMqSVEdO/cnc5PPF
S6tYiGaEoSxJdWTs3O+U/dZStTo0M9x9LUk1pEj0r+RiMQ/AWPzYUz6/FYC+2U+tTmFKhZ2yJNWQ
EMZThQmgFMat8XuQ1dgMZUmqISGMM7noncghjF/SdQIAPy16WlcjM5

In [46]:
f = Formula(nothing, :(0+category))
mm_full = reduce(vcat, ModelMatrix(ModelFrame(f, x)).m for x in nested_by_run[:data])

222×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 ⋮       
 0.0  1.0
 0.0  1.0
 1.0  0.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 0.0  1.0
 1.0  0.0
 1.0  0.0

In [47]:
ModelMatrix(ModelFrame(f, behav)).m == mm_full

true

In [48]:
k = [:run_number, :subject_num]
join(run_niftis[reverse(1:end), k], nested_by_run[reverse(1:end), :], on=k, kind=:semi)

,run_number,subject_num
1,4,3
2,3,3
3,2,3
4,1,3


In [49]:

join(nested_by_run, run_niftis[reverse(1:end), k], on=k)

,run_number,subject_num,data
1,1,3,"55×22 DataFrames.SubDataFrame{Array{Int64,1}}. Omitted printing of 19 columns │ Row │ experiment_name │ expt_start_date_time │ subject_num │ ├─────┼─────────────────┼──────────────────────┼─────────────┤ │ 1 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 2 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 3 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 4 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 5 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 6 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 7 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 8 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 9 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 10 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 11 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ ⋮ │ 44 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 45 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 46 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 47 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 48 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 49 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 50 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 51 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 52 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 53 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 54 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 55 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │"
2,2,3,"56×22 DataFrames.SubDataFrame{Array{Int64,1}}. Omitted printing of 19 columns │ Row │ experiment_name │ expt_start_date_time │ subject_num │ ├─────┼─────────────────┼──────────────────────┼─────────────┤ │ 1 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 2 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 3 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 4 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 5 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 6 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 7 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 8 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 9 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 10 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 11 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ ⋮ │ 45 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 46 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 47 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 48 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 49 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 50 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 51 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 52 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 53 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 54 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 55 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 56 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │"
3,3,3,"56×22 DataFrames.SubDataFrame{Array{Int64,1}}. Omitted printing of 19 columns │ Row │ experiment_name │ expt_start_date_time │ subject_num │ ├─────┼─────────────────┼──────────────────────┼─────────────┤ │ 1 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 2 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 3 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 4 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 5 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 6 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 7 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 8 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 9 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 10 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 11 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ ⋮ │ 45 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 46 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 47 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 48 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 49 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 50 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 51 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 52 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 53 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 54 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 55 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │ │ 56 │ vot_adapt │ 30-Apr-2015 13:53:51 │ 3 │"
4,4,3,"55×22 DataFrames.SubDataFrame{Array{Int64,1}}. Omitted printing of 19 columns │ Row │ experiment_name │ expt_start_date_time │ subject_num │ ├─────┼─────────────────┼─────────

In [50]:
Base.cat(catdim::Integer, X::NIVolume...) = NIVolume(X[1].header, cat(catdim, (ni.raw for ni in X)...))

nifti_combined = cat(4, run_niftis[:nifti]...)
@assert size(nifti_combined, 4) == sum(size(ni, 4) for ni in run_niftis[:nifti])

In [51]:
headers = [ni.header for ni in run_niftis[:nifti]]
getaffine.(headers)

4-element Array{Array{Float64,2},1}:
 [-4.0 -1.00259e-21 7.86008e-21 130.034; 2.27145e-20 -4.0 -7.2096e-20 155.757; 1.90928e-20 6.87114e-20 4.0 -57.6159; 0.0 0.0 0.0 1.0]        
 [-3.99997 0.000303863 0.0165243 129.695; -0.000619086 -3.99927 -0.0763172 156.701; 0.0165155 -0.0763191 3.99924 -54.5768; 0.0 0.0 0.0 1.0]  
 [-3.99993 0.0230993 0.000149058 128.603; -0.0230996 -3.99969 -0.044128 157.424; -0.000105785 -0.0441281 3.99976 -56.4733; 0.0 0.0 0.0 1.0]  
 [-3.99999 0.00438923 -0.00662023 129.765; -0.00436476 -3.99997 -0.0147656 156.017; -0.00663638 -0.0147584 3.99997 -56.5732; 0.0 0.0 0.0 1.0]

...of course they're different. it's the non-preprocessed data, not normalized.  this is fine for now but I'll want to add a check in the real thing.

In [37]:
nested_by_run[k]
run_niftis[k]

zip((nested_by_run[kk] for kk in k)...)

Base.Zip2{DataArrays.DataArray{Int64,1},DataArrays.DataArray{Int64,1}}([1,2,3,4],[3,3,3,3])

# Frame type

This type holds the events DataFrame, the BOLD images/NIVolumes, and the mapper between them.  Define methods on this field to create model matrices and fit models with a formula.

## Fields

* event_time_col::Symbol column in events with the event times (in seconds)
* events::Vector{AbstractDataFrame} Events for each NIVolume, with column eventtimecol
* nivolumes::Vector{NIVolume} Vector of NIVolumes
* tr_to_event::Matrix transforms between events and TRs (e.g., the HRF for each event)

In [ ]:
type BOLDFrame
    events::AbstractVector{AbstractDataFrame}
    nivolumes::AbstractVector{NIVolume}
    event_time_col::Symbol
    tr_to_event::Matrix
end



In [57]:

function BOLDFrame{T,U,V}(events::AbstractVector{AbstractDataFrame}, nivolumes::AbstractVector{NIVolume{T,U,V}}; 
                          event_time_col::Symbol=:time, hrf::HRF=canonical_hrf)
    length(events) == length(nivolumes) || error("Mismatch in number of volumes and events")
    tr_to_event = blkdiag(make_tr_to_event_mat.(nivolumes, events, [event_time_col], hrf=hrf)...)
    BOLDFrame(events, nivolumes, event_time_col, tr_to_event)
end

BOLDFrame

In [60]:
bf = BOLDFrame(nested_by_run[:data], run_niftis[:nifti], event_time_col=:start_secs)
bf.tr_to_event

464×222 Array{Float64,2}:
  0.0           0.0           0.0          …   0.0           0.0       
  0.101658      0.0           0.0              0.0           0.0       
  0.159794      7.93649e-11   0.0              0.0           0.0       
  0.0564006     0.103331      0.0              0.0           0.0       
  0.000158514   0.159103      2.47697e-9       0.0           0.0       
 -0.0152137     0.0556812     0.104996     …   0.0           0.0       
 -0.0127044    -9.58782e-5    0.1584           0.0           0.0       
 -0.00639594   -0.0152424     0.0549664        0.0           0.0       
 -0.0023371    -0.0126534    -0.000347576      0.0           0.0       
 -0.000672901  -0.00635017   -0.0152698        0.0           0.0       
 -0.000160727  -0.00231515   -0.0126022    …   0.0           0.0       
  0.0          -0.000665426  -0.00630458       0.0           0.0       
  0.0          -0.000158721  -0.00229337       0.0           0.0       
  ⋮                                   

# Modeling

## Fixed effects model matrix

This is very straingtforward.  Just generate the individual model matrices, concatenate them, and multiple by the transformer.

Makes me wonder whether it's worth having one big transformation vs. a bunch of little ones (one per pair).  Easy enough to benchmark once I know the general strategy works

In [76]:
function model_matrix(f::Formula, bf::BOLDFrame)

    g = Formula(nothing, f.rhs)

    bf.tr_to_event * vcat((ModelMatrix(ModelFrame(g, events)).m for events in bf.events)...)
end

model_matrix(@formula(_ ~ 0 + category + (1|subject_num)), bf)

464×2 Array{Float64,2}:
  0.0           0.0        
  0.0           0.101658   
  7.93649e-11   0.159794   
  0.103331      0.0564006  
  0.159103      0.000158517
  0.0556812     0.089782   
 -9.58782e-5    0.145696   
 -0.0152424     0.0485705  
 -0.0126534     0.104792   
 -0.00635017    0.249683   
 -0.00231493    0.198105   
  0.10927       0.0465265  
  0.156073      0.0923046  
  ⋮                        
  0.0147794    -0.00704367 
 -0.0254105     0.156515   
 -0.0254667     0.110626   
 -0.0143377     0.0330021  
 -0.0057665     0.157288   
 -0.00180203    0.0981643  
 -0.000461619   0.0144483  
 -8.41892e-5   -0.0143349  
  0.0          -0.0167093  
  0.0          -0.0101895  
  0.0          -0.00431315 
  0.0          -0.00137992 

## Random effects model matrices

MixedModels uses a special `[Scaler|Vector]ReMat` type, which keeps the model matrix and the group indices separately, and has special methods to do linear algebra.  This has the advantage (I think) of using an efficient storage format.  But it will happily fit with a regular sparse/dense matrix, and on the dyestuff data there's very little difference between these three.  The only problem comes from printing the output (and generating the `VarCorr` summary), since the `ReMat` types also keep track of their grouping factor name.

It's not possible to use the normal ReMat types because the HRFs for, say, stimulus types might overlap, so you can't simply say which level each row of the model matrix applies to.  There are two options as I see them:

1. Fit using a "naked" matrix type and don't worry about plugging in the names until after the fact (since the fitting is going to happen internally anyway, with the results copied into some storage format...)
2. Create another `ReMat` subtype that stores the whole matrix and defines/delegates the same methods that MixedModels ReMats do (some linear algebra and size and levels).

I think the second strategy is better long term, but setting up all the methods might be annoying since there are a lot of specialized methods. At least there's a guide to what needs to be covered in the MixedModels.

Internally, a mixed model stores the random effects matrices, fixed effects matrix, and the response in a vector in the `.trms` field.  S

In [97]:
rem = MixedModels.remat(:((1 | run_number)), vcat(bf.events...))

MixedModels.ScalarReMat{Float64,Int64,UInt8}([1,1,1,1,1,1,1,1,1,1  …  4,4,4,4,4,4,4,4,4,4],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0  …  1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0],:run_number,String["(Intercept)"])

In [101]:
size(rem)

(222,4)

In [100]:
rem.f.refs

222-element Array{UInt8,1}:
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
    ⋮
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04